# **AutoML com AutoKeras**
- O Automated Machine Learning é também conhecido como AutoML;
- Ferramentas e serviços que abstraem os detalhes e conhecimentos necessários para se realizar ML/DL;
- Automatiza-se determinadas tarefas necessárias no desenvolvimento de algum modelo;
- Geralmente, cobrem etapas de normalização de dados e engenharia de “features”; treinamento de modelos de diferentes tipos e com diferentes hiperparâmetros; avaliação e comparação de resultados;
- Algumas ferramentas: H2O, AutoKeras e TPOT;

- Vantagens:  
  - Necessário conhecimento superficial de ML/DL para se utilizar as ferramentas;
  - Diminui o tempo para se ter resultados iniciais;
  - Pode-se utilizar em diferentes tipos de dados e problemas (baseline);
  - Algumas ferramentas possibilitam customizar modelos/layers;
- Desvantagens:
  - Customização que pode ser limitada ou dificultada;
  - Pode-se não obter resultado bom o bastante para o problema, precisando da intervenção de especialistas;



##**Instalação da biblioteca AutoKeras**

In [ ]:
%%capture
#!pip install autokeras

##**Import dos pacotes necessários**

In [1]:
import os
import sklearn
import tensorflow
import pandas as pd
import autokeras as ak

from tensorflow import keras
from keras import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

##**Cópia do dataset**

In [2]:
%%time
!rmdir /S /Q lectures-cdas-2023
!git clone https://github.com/renansantosmendes/lectures-cdas-2023.git

O sistema n�o pode encontrar o arquivo especificado.


CPU times: total: 31.2 ms
Wall time: 2.92 s


Cloning into 'lectures-cdas-2023'...


##**Leitura do dataset em um DataFrame do Pandas**

In [3]:
data = pd.read_csv('lectures-cdas-2023/fetal_health.csv')

In [4]:
import shutil
shutil.rmtree('Auto_model', ignore_errors=True)

##**Processamento dos dados**
Nesta célula serão feitos os seguintes passos:
- Separação das variáveis de entrada e saída
- Padronização dos valores das variáveis de entrada
- Separação em dados de treino e teste
- Categorização dos dados de saída (a serem preditos)

In [5]:
X=data.drop(["fetal_health"], axis=1)
y=data["fetal_health"]

columns_names = list(X.columns)
X_df = StandardScaler().fit_transform(X)
X_df = pd.DataFrame(X_df, columns=columns_names)

X_train, X_test, y_train, y_test = train_test_split(X_df, y, test_size=0.3, random_state=42)
y_train=keras.utils.to_categorical(y_train)[:,1:]
y_test=keras.utils.to_categorical(y_test)[:,1:]

##**Criação do modelo (atualizado)**
A partir da versão 2.0.0 do AutoKeras a classe StructuredDataClassifier foi descontinuada e uma forma de contornar é por meio do uso da classe AutoModel em conjunto com a ClassificationHead. Dessa forma o modelo poderá ser treinado conforme o código abaixo

In [7]:
input_node = ak.Input()
output_node = ak.ClassificationHead(num_classes=3)(input_node)
auto_model = ak.AutoModel(
    inputs=input_node,
    outputs=output_node,
    max_trials=15,
    overwrite=True,
    directory='Auto_model'
)

auto_model.fit(X_train.values, y_train, epochs=10)

Trial 15 Complete [00h 00m 01s]
val_loss: 0.25644537806510925

Best val_loss So Far: 0.21382062137126923
Total elapsed time: 00h 00m 23s
Epoch 1/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 992us/step - accuracy: 0.7570 - loss: 0.6518 
Epoch 2/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8841 - loss: 0.2867 
Epoch 3/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 957us/step - accuracy: 0.8908 - loss: 0.2636
Epoch 4/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 955us/step - accuracy: 0.8930 - loss: 0.2512
Epoch 5/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8952 - loss: 0.2417
Epoch 6/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 937us/step - accuracy: 0.8946 - loss: 0.2341
Epoch 7/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 899us/step - accuracy: 0.8970 - loss: 0.2269
Epoch 8/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 900us/step - accuracy: 0.9026 - loss: 0.2198
Epoch 9/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 892us/step - accuracy: 0.9069 - loss: 0.2138
Epoch 10/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9110 - loss: 0.2091


##**Exportando o melhor modelo encontrado até o momento**

In [8]:
model = auto_model.export_model()

c:\Users\vitor\anaconda3\envs\puc-mlops-ead-environment\lib\site-packages\keras\src\saving\saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adamw', because it has 1 variables whereas the saved optimizer has 5 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


##**Vendo o resumo do melhor modelo**

In [9]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 21)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ cast_to_float32 (CastToFloat32) │ (None, 21)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 3)              │            66 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ classification_head_2 (Softmax) │ (None, 3)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 66 (264.00 B)

 Trainable params: 66 (264.00 B)

 Non-trainable params: 0 (0.00 B)

##**Avaliando o modelo encontrado com os dados de teste**

In [10]:
model.evaluate(X_test, y_test)

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8842 - loss: 0.2807  


[0.2685399651527405, 0.8855799436569214]